In [ ]:
import pandas as pd

In [ ]:
csv_columns = ['PRIMARIO', 'FUERA DE SERVICIO', 'REPOSICION', 'TIPO DE EVENTO', 'ESTADO DEL CLIMA', 'OBSERVACION']

In [ ]:
# leer archivos
df1 = pd.read_csv(r"../data/202001.csv", encoding="ISO-8859-1")

In [ ]:
# Transformar datasets
df1n = df1.BARRIOS.str.rstrip(',').str.split(',').apply(pd.Series)    # break df.BARRIOS into columns
df1n.index = df1.set_index(csv_columns).index
df1 = df1n.stack().reset_index(csv_columns)    # stack and reset_index
df1.columns = csv_columns  + ['BARRIO']

In [ ]:
df1['DURACION'] = df1[["FUERA DE SERVICIO", "REPOSICION"]].apply(lambda x: (pd.to_datetime(x[1], format="%d/%m/%Y %H:%M", errors='coerce') - pd.to_datetime(x[0], format="%d/%m/%Y %H:%M", errors='coerce')), axis=1)

In [ ]:
# Normalizar BARRIO
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("\r", " "))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.strip())
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Hiódromo.", "Hipódromo"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Madame Lych", "Madame Lynch"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Dr. Gaspar R. de Francia", "Dr. Gaspar Rodriguez de Francia"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mcal. Francisco.S. López", "Mcal. Francisco. S. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mbocayty", "Mbocayaty"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Cañada de Ybyray", "Cañada Ybyray"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazaret", "Nazareth"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazarethh", "Nazareth"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Carlos. A. López", "Carlos A. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.rstrip("."))


In [ ]:
# Agregar Duración en Segundos
df1['DURACION SEGS'] = df1['DURACION'].apply(lambda x: x.seconds)

In [ ]:
df1

In [ ]:
aux = list(df1['BARRIO'].unique())

In [ ]:
AUX = df1.groupby(['TIPO DE EVENTO', 'BARRIO'])['BARRIO'].count()

In [ ]:
AUX[["Accidental"]].sort_values(ascending=False)

In [ ]:
AUX[["Programado"]].sort_values(ascending=False)

In [ ]:
aux3 = df1[['BARRIO', 'DURACION']]

In [ ]:
aux4 = df1.sort_values(by=['DURACION'], ascending=False)

In [ ]:
aux4.sort_values(by=["DURACION"], ascending=False)

In [ ]:
aux4.loc[(aux4["REPOSICIONN"]=="Recierre") & (aux4["FUERA DE SERVICIO"].str.contains("10/01/2020"))]

In [ ]:
aux4.loc[(aux4["FUERA DE SERVICIO"].str.contains("31/01/2020"))]

In [ ]:
tdelta2 = pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M") - pd.to_datetime("6/01/2020 23:54", format="%d/%m/%Y %H:%M")
tdelta2

In [ ]:
pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M")